<img src="Quantum_Brilliance_dark_blue_Logo_RGB.png" alt="Copyright (c) 2021 Quantum Brilliance Pty Ltd" width="240">

# qbOS Build 0.0721.002
> *Quantum Brilliance Operating System supporting a complete set of quantum computational tasks*

## Bernstein-Vazirani Algorithm

This algorithm works with the function, `f`:

$ \hspace{1cm} f_s(x) = (s \cdot x ) \text{ mod 2} $

$ \text{where:} $

$ s \text{ is a fixed "secret" bitstring} $

$ x \text{ is a bitstring query that is an attempt to discover the "secret" }$

In classical computing, discovering the secret would require `n` queries, where `n` is the number of bits in the secret string:

$ \hspace{1cm} x = 2^i, \;\; \forall i \; \in \; [0, n-1] $

By contrast the Bernstein-Vazirani quantum algorithm requires only 1 call to `f`.

In [1]:
import xacc
import sys
import re
import statistics as stat
import qbos

### User inputs

In [2]:
min_qubits = 14       # Lower bound for a sweep over #qubits in the secret, s 
max_qubits = 16       # Upper bound for a sweep over #qubits in the secret, s
repetitions = 1

acc_string = 'aer'    # aer is required for noise-enabled results

### Bernstein-Vazirani experiment design

| Bernstein-Vazirani | Conditions | Noise: disabled | Noise: enabled |
| --- | --- | --- | --- |
| | Common to all: shots = 256 |  |  |
| 14-qubit secret | QPU Kernel: `gen_qbstrBV(14)` | `Acquire shot counts from aer simulation` | `Acquire shot counts from aer simulation` |
| 15-qubit secret | QPU Kernel: `gen_qbstrBV(15)` | `Acquire shot counts from aer simulation` | `Acquire shot counts from aer simulation` |
| 16-qubit secret | QPU Kernel: `gen_qbstrBV(16)` | `Acquire shot counts from aer simulation` | `Acquire shot counts from aer simulation` |

### Circuit generator for Bernstein-Vazirani algorithm, secret = "1010...10"

In [3]:
def gen_qbstrBV(nb_qubits):
    output = nb_qubits - 1
    # Generate oracle
    oracle = [qubit for qubit in range(output) if qubit%2 ]
    
    ## qasm input
    prefix = '''.compiler xasm\n.circuit BV_example\n.parameters x\n.qbit q'''
    
    # Initialisation
    initialisation = '''\n// Initialise\n'''
    for qubit in range(output+1):
        initialisation += '''H(q[%i]);\n''' % qubit
    initialisation += '''Z(q[%i]);\n''' % output
    
    # Oracle
    oracle_string = '''\n// Oracle\n'''
    for qubit in oracle:
        oracle_string += '''CNOT(q[%i],q[%i]);\n''' % (qubit,output)
        
    # Unentangle
    unentangle = '''\n// Unentangle\n'''
    for qubit in range(output+1):
        unentangle += '''H(q[%i]);\n''' % qubit
        
    # Measurement
    measurement = '''\n// Measurement\n'''
    for qubit in range(output):
        measurement += '''Measure(q[%i]);\n''' % qubit
    generator = prefix + initialisation + oracle_string + unentangle + measurement

    # Convert input string
    xacc.qasm(generator)
    BV_example = xacc.getCompiled('BV_example')
    sdstaq = xacc.getCompiler('staq')
    q2oqm = re.sub(r"qreg q\[\d+\];", "", sdstaq.translate(BV_example))
    return '__qpu__ void QBCIRCUIT(qreg q) {\n' + q2oqm + '}'
    

## Setup all conditions for the experiment

In [4]:
range_qubits = list(range(min_qubits+1,max_qubits+2))
sweepstr = [gen_qbstrBV(el) for el in range_qubits for rep in range(repetitions)]
qubit_range = [qubit for qubit in range_qubits for rep in range(repetitions)]

In [5]:
tqb=qbos.core()
tqb.qb12()

# Back end simulator
tqb.acc=acc_string

# Number of shots
tqb.sn[0].clear()
sweep = [256]
[tqb.sn[0].append(bb) for bb in sweep]

# Noise enabled
tqb.noise[0].clear()
sweep = [False,True]
[tqb.noise[0].append(bb) for bb in sweep]

# Circuits
tqb.instrings.clear()
tqb.qns.clear()
for nb_index in range(len(qubit_range)):
    tqb.qns.append(qbos.N()) 
    tqb.qns[nb_index].append(1+qubit_range[nb_index])
    tqb.instrings.append(qbos.String()) 
    tqb.instrings[nb_index].append(sweepstr[nb_index])

## Execute the quantum circuits

In [6]:
tqb.run()

## Results for 14-qubit secret

### Noise-free

In [7]:
print('Noise-free counts:')
print(tqb.out_raw[0][0])

Noise-free counts:
{
    "10101010101010": 256
}


### Noise-enabled 

In [8]:
print('Noisy counts:')
print(tqb.out_raw[0][1])

Noisy counts:
{
    "00101010101010": 3,
    "10001010101010": 4,
    "10100010001010": 1,
    "10100010101010": 2,
    "10101000101010": 2,
    "10101000111010": 1,
    "10101001101010": 1,
    "10101010001010": 1,
    "10101010100010": 3,
    "10101010101000": 3,
    "10101010101010": 226,
    "10101010101110": 3,
    "10101010111010": 1,
    "10101011101010": 1,
    "10101011101011": 1,
    "10101110101010": 1,
    "10111010101010": 1,
    "11101010101011": 1
}


## Result for 16-qubit secret

### Noise-free

In [9]:
print('Noise-free counts:')
print(tqb.out_raw[2][0])

Noise-free counts:
{
    "1010101010101010": 256
}


### Noise-enabled

In [10]:
print('Noisy counts:')
print(tqb.out_raw[2][1])

Noisy counts:
{
    "0010101010101010": 1,
    "1000101010101010": 5,
    "1010001010101010": 3,
    "1010100010101010": 2,
    "1010101000101010": 2,
    "1010101010001010": 5,
    "1010101010100010": 1,
    "1010101010101000": 2,
    "1010101010101010": 214,
    "1010101010101011": 4,
    "1010101010101110": 1,
    "1010101010111010": 4,
    "1010101011101010": 1,
    "1010101110101010": 2,
    "1010101110101011": 1,
    "1010111010100010": 1,
    "1010111010101010": 3,
    "1011101010101010": 3,
    "1110101010101010": 1
}
